In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
os.getcwd()

'C:\\users\\keele\\onedrive\\learning\\udacity courses\\202103 - Data Scientist\\Part 1\\Medium Post Project'

In [41]:
df_2018 = pd.read_csv('datasets/2018.csv')
df_2020 = pd.read_csv('datasets/2020.csv')

## Part 1: Finding a Year Worth Contrasting with 2020

1. 2011: No-go. Surveys were administered for two completely different purposes. 2011 looks like it's meant to help them find advertisers.
2. 2012: Same
3. 2013: Same
4. 2014: Same
5. 2018: Looks good. Respondent, country, major, devtype, gender, age, languages, databases, platforms all shared.

In [42]:
# First, figure out which fields they have in common
shared_cols = []
for col in df_2018.columns:
    if col in df_2020.columns:
        print(col)
        shared_cols.append(col)

Respondent
Country
Employment
UndergradMajor
DevType
CurrencySymbol
LanguageWorkedWith
LanguageDesireNextYear
DatabaseWorkedWith
DatabaseDesireNextYear
PlatformWorkedWith
PlatformDesireNextYear
Gender
Age


## Part 2: Narrow down dataframes to common fields

In [50]:
df_2018_subset = df_2018[shared_cols]
df_2020_subset = df_2020[shared_cols]

In [51]:
df_2018_subset

,Respondent,Country,Employment,UndergradMajor,DevType,CurrencySymbol,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,Gender,Age
0,1,Kenya,Employed part-time,Mathematics or statistics,Full-stack developer,KES,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Male,25 - 34 years old
1,3,United Kingdom,Employed full-time,"A natural science (ex. biology, chemistry, phy...",Database administrator;DevOps specialist;Full-...,GBP,JavaScript;Python;Bash/Shell,Go;Python,Redis;PostgreSQL;Memcached,PostgreSQL,Linux,Linux,Male,35 - 44 years old
2,4,United States,Employed full-time,"Computer science, computer engineering, or sof...",Engineering manager;Full-stack developer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,United States,Employed full-time,"Computer science, computer engineering, or sof...",Full-stack developer,NaN,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,"SQL Server;Microsoft Azure (Tables, CosmosDB, ...","SQL Server;Microsoft Azure (Tables, CosmosDB, ...",Azure,Azure,Male,35 - 44 years old
4,7,South Africa,Employed full-time,"Computer science, computer engineering, or sof...",Data or business analyst;Desktop or enterprise...,ZAR,C;C++;Java;Matlab;R;SQL;Bash/Shell,Assembly;C;C++;Matlab;SQL;Bash/Shell,SQL Server;PostgreSQL;Oracle;IBM Db2,PostgreSQL;Oracle;IBM Db2,Arduino;Windows Desktop or Server,Arduino;Windows Desktop or Server,Male,18 - 24 years old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98850,101513,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98851,101531,Spain,"Not employed, but looking for work",NaN,Back-end developer;Front-end developer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98852,101541,India,Employed full-time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98853,101544,Russian Federation,"Independent contractor, freelancer, or self-em...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 3: Come up with 3 business questions worth asking

Theme: US vs India
1. How has language popularity changed since 2018?
2. Which variable tends to influence number of languages known: country, employment, major or devtype?
3. Is languageDesiredNextYear a good indicator for languagesWorkedWith in the future?

Relevance to Business: What kinds of people are easiest to hire in the two regions?

## Part 4: Narrow down the dataset to US and India respondents

In [52]:
relevant_countries = ['United States','India']

In [54]:
df_2020_usin = df_2020_subset[df_2020_subset['Country'].isin(relevant_countries)].copy()
df_2018_usin = df_2018_subset[df_2018_subset['Country'].isin(relevant_countries)].copy()

In [56]:
df_2020_usin.shape

(20872, 14)

In [57]:
df_2018_usin.shape

(34030, 14)

34,000 responses in 2018 contrasts with 21,000 responses in 2020.

## Part 5: Clean the datasets

In [85]:
# Figure out how nan affects our datasets
null_2018 = df_2018_usin.isnull()
null_2020 = df_2020_usin.isnull()

null_counts = []

for col in null_2018.columns:
    null_dict = {}
    null_dict['column'] = col
    null_dict['2018_nulls'] = round(null_2018[col].mean(),2)
    null_dict['2020_nulls'] = round(null_2020[col].mean(),2)
    null_dict['2018_vals'] = len(null_2018[col]) - null_2018[col].sum()
    null_dict['2020_vals'] = len(null_2020[col]) - null_2020[col].sum()
    null_counts.append(null_dict)

In [86]:
pd.DataFrame(null_counts).sort_values(ascending = True, by = '2020_nulls')

,column,2018_nulls,2020_nulls,2018_vals,2020_vals
0,Respondent,0.00,0.00,34030,20872
1,Country,0.00,0.00,34030,20872
2,Employment,0.03,0.01,32973,20740
6,LanguageWorkedWith,0.20,0.11,27351,18517
7,LanguageDesireNextYear,0.25,0.17,25441,17335
3,UndergradMajor,0.15,0.18,28801,17094
10,PlatformWorkedWith,0.33,0.18,22667,17165
4,DevType,0.06,0.23,31907,15980
11,PlatformDesireNextYear,0.38,0.23,21105,16148
8,DatabaseWorkedWith,0.32,0.24,23100,15897


Every field seems more or less useful from a distance. Question remains about breaking the dataset down into segments. Will there be enough Indian and US respondents?

Languages fields are especially complete. This is good, since the primary business questions focus on languages.